# Import Libraries

In [2]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet_v2 import decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image

from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import AdversarialPatch

# Load Data

In [ ]:
image_path = "racer.jpg"
img = image.load_img(image_path, target_size=(224, 224))

In [4]:
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)  
img = np.clip(img, 0, 1)

# Model

In [5]:
model = ResNet50V2(weights="imagenet")

In [6]:
model.compile(
    optimizer=Adam(),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Adversarial Patch

In [8]:
mean = [103.939, 116.779, 123.68]

In [9]:
classifier = TensorFlowV2Classifier(
    model=model,
    loss_object=None,
    train_step=None,
    input_shape=(224, 224, 3),
    clip_values=(0, 255),
    nb_classes=1000,
    preprocessing=(
        mean, np.array([1.0, 1.0, 1.0])
    )
)

In [10]:
adversarial_patch = AdversarialPatch(
    classifier=classifier,
    rotation_max=22.5,
    scale_min=0.4,
    scale_max=1.0,
    learning_rate=5000.,
    max_iter=500,
    batch_size=16,
    patch_shape=(224, 224, 3)
)

In [11]:
target_class = 10

In [13]:
y_one_hot = np.zeros(1000)
y_one_hot[target_class] = 1.0

In [14]:
y_target = np.expand_dims(y_one_hot, axis=0)

In [ ]:
patch, patch_mask = adversarial_patch.generate(x=img, y=y_target)

In [16]:
patched_image = adversarial_patch.apply_patch(img, scale=0.5)

# Results

In [23]:
def predict_image(img):
    pred = model.predict(img, verbose=0)
    decoded_preds = decode_predictions(pred, top=5)[0]
    return {"class_name": decoded_preds[0][1], "confidence_score": round(decoded_preds[0][2], 2)}

In [24]:
print("Raw Image Prediction:", predict_image(img))
print("Adversarial Image Prediction:", predict_image(patched_image))

Raw Image Prediction: {'class_name': 'racer', 'confidence_score': 0.73}
Adversarial Image Prediction: {'class_name': 'chambered_nautilus', 'confidence_score': 1.0}
